## Perfomance~ 

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from functools import partial
from collections import namedtuple
from itertools import chain

# 작업의 시작 및 종료 시간을 포함하는 이벤트를 나타내는 namedtuple
Event = namedtuple('Event', 'job start end')

# 작업 실행 비용 함수
def compcost(job, agent):
    return exec_times_gpu[job][agent]

    
# 통신 비용 함수
def commcost(nj, ni, agent_from, agent_to):
    if agent_from == agent_to:
        return 0
    else:
        if(ni=='1' and nj=='2'):
            return 18
        if(ni=='1' and nj=='3'):
            return 12
        if(ni=='1' and nj=='4'):
            return 9
        if(ni=='1' and nj=='5'):
            return 11
        if(ni=='1' and nj=='6'):
            return 14
        if(ni=='2' and nj=='8'):
            return 19
        if(ni=='2' and nj=='9'):
            return 16
        if(ni=='3' and nj=='7'):
            return 23
        if(ni=='4' and nj=='8'):
            return 27
        if(ni=='4' and nj=='9'):
            return 23
        if(ni=='5' and nj=='9'):
            return 13
        if(ni=='6' and nj=='8'):
            return 15
        if(ni=='7' and nj=='10'):
            return 17
        if(ni=='8' and nj=='10'):
            return 11
        if(ni=='9' and nj=='10'):
            return 13
        else:
            return 0

# DAG (작업 그래프) 정의
dag = {
    '1': ['2', '3', '4', '5', '6'],
    '2': ['8', '9'],
    '3': ['7'],
    '4': ['8', '9'],
    '5': ['9'],
    '6': ['8'],
    '7': ['10'],
    '8': ['10'],
    '9': ['10'],
}
    

# 각 GPU마다 존재하는 실행 시간 상대적인 값
exec_times_gpu = {
    '1': [14, 16, 9],
    '2': [13, 19, 18],
    '3': [11, 13, 19],
    '4': [13, 8, 17],
    '5': [12, 13, 10],
    '6': [13, 16, 9],
    '7': [7, 15, 11],
    '8': [5, 11, 14],
    '9': [18, 12, 20],
    '10': [21, 7, 16]
}



In [2]:
# 평균 실행 시간 계산 함수 (소숫점 첫째 자리까지 반올림)
def calculate_avg_exec_times(exec_times_gpu):
    avg_exec_times = {}
    for node, times in exec_times_gpu.items():
        avg_exec_times[node] = round(sum(times) / len(times), 1)  # 소숫점 첫째 자리까지 반올림
        # print(avg_exec_times[node])
    return avg_exec_times

avg_exec_times = calculate_avg_exec_times(exec_times_gpu)

# Upward Rank 계산 함수
def calculate_ranku(dag, exec_times):
    rank = {}
    def ranku(node):
        if node in rank:
            return rank[node]
        if node not in dag or not dag[node]:
            rank[node] = exec_times[node]
        else:
            rank[node] = exec_times[node] + max(commcost(succ, node, 0, 1) + ranku(succ) for succ in dag[node])
        return rank[node]
    for node in dag:
        ranku(node)
    return rank

# 우선순위 계산
ranks = calculate_ranku(dag, avg_exec_times)

# 우선순위 출력
print("우선순위 (Upward Rank):")
for node, rank_value in sorted(ranks.items(), key=lambda x: x[1], reverse=True):
    print(f"{node}: {rank_value}")


우선순위 (Upward Rank):
1: 108.1
4: 80.10000000000001
3: 80.0
2: 77.1
5: 69.1
6: 63.400000000000006
9: 44.4
7: 42.7
8: 35.7
10: 14.7


In [3]:
def schedule_tasks(dag, exec_times_gpu, num_agents):
    """
    작업을 각 에이전트에 효율적으로 스케줄링하는 함수

    Args:
        dag (dict): 작업 간의 의존성을 나타내는 DAG
        exec_times_gpu (dict): 각 작업이 각 에이전트에서 실행되는 시간
        num_agents (int): 사용 가능한 에이전트의 수

    Returns:
        dict: 각 에이전트에 스케줄링된 작업들의 목록
    """
    
    # 1. 각 작업의 우선순위를 계산
    avg_exec_times = calculate_avg_exec_times(exec_times_gpu)
    rank = calculate_ranku(dag, avg_exec_times)
    
    # 2. 우선순위에 따라 작업을 내림차순으로 정렬
    sorted_tasks = sorted(rank, key=rank.get, reverse=True)
    print("Sorted tasks based on priority:", sorted_tasks)

    # 3. 각 에이전트별로 스케줄을 초기화
    schedules = {agent: [] for agent in range(num_agents)}
    
    # 4. 각 에이전트의 다음 사용 가능 시간을 0으로 초기화
    agent_available_time = {agent: 0 for agent in range(num_agents)}
    
    # 5. 각 작업이 어느 에이전트에서 언제 완료되었는지를 추적하는 딕셔너리 초기화
    task_assignment = {}
    
    # 6. 각 작업의 선행 작업을 찾기 위한 역방향 DAG 생성
    reverse_dag = {task: [] for task in dag}
    for task, successors in dag.items():
        for succ in successors:
            if succ not in reverse_dag:
                reverse_dag[succ] = []
            reverse_dag[succ].append(task)

    # 7. 우선순위가 높은 작업부터 순차적으로 스케줄링
    for task in sorted_tasks:
        print(f"\nScheduling task: {task}")
        earliest_start_time = float('inf')  # 현재 작업의 가장 이른 시작 시간을 무한대로 초기화
        best_agent = None  # 현재 작업을 가장 빨리 실행할 수 있는 에이전트를 저장할 변수

        # 선행 작업들을 확인 
        successors = reverse_dag.get(task, [])
        print(f"Task {task} successors: {successors}")
        
        # 모든 에이전트를 순회하며 가장 효율적인 에이전트를 찾음
        for agent in range(num_agents):
            succ_times = []
            # 후속 작업들에 대한 통신 비용을 모두 고려함
            for succ in successors:
                succ_agent, succ_finish_time = task_assignment.get(succ, (None, 0))
                
                # 후속 작업이 스케줄링되지 않았으면, 현재 작업의 통신 비용 고려
                if succ_agent is None:
                    continue

                # 만약 후속 작업이 다른 에이전트에서 실행되었다면 통신 비용 적용
                comm_time = commcost(task, succ, agent, succ_agent) if succ_agent != agent else 0
                print(f"Comm time between task {task} on agent {agent} and task {succ} on agent {succ_agent}: {comm_time}")
                succ_times.append(succ_finish_time + comm_time)

            # 후속 작업이 있는 경우만 시작 시간을 계산
            if succ_times:
                start_time = max(
                    agent_available_time[agent],  # 해당 에이전트가 사용 가능해지는 시간
                    max(succ_times, default=0)  # 후속 작업의 완료 시간 + 통신 시간 중 가장 늦은 값
                )
            else:
                start_time = agent_available_time[agent]  # 후속 작업이 없을 경우 바로 시작 가능

            print(f"Start time for task {task} on agent {agent}: {start_time}")
            
            # 해당 에이전트에서 작업을 완료하는 시간 계산
            finish_time = start_time + exec_times_gpu[task][agent]
            print(f"Finish time for task {task} on agent {agent}: {finish_time}")
            
            # 가장 빠른 완료 시간을 가진 에이전트와 시간을 저장
            if finish_time < earliest_start_time:
                earliest_start_time = finish_time
                best_agent = agent
                print(f"Best agent updated to {best_agent} with earliest finish time {earliest_start_time}")
        
        # 선택된 에이전트의 스케줄에 현재 작업 추가
        schedules[best_agent].append((task, agent_available_time[best_agent], earliest_start_time))
        print(f"Task {task} scheduled on agent {best_agent} from {agent_available_time[best_agent]} to {earliest_start_time}")
        
        # 해당 에이전트의 다음 사용 가능 시간을 업데이트
        agent_available_time[best_agent] = earliest_start_time
        print(f"Agent {best_agent} available at {agent_available_time[best_agent]}")
        
        # 현재 작업의 에이전트 할당 및 완료 시간을 저장
        task_assignment[task] = (best_agent, earliest_start_time)
    
    return schedules


In [4]:
# 예시 실행
if __name__ == "__main__":
    num_agents = 3
    schedules = schedule_tasks(dag, exec_times_gpu, num_agents)  
    
    print("Scheduled Tasks:")
    for agent, tasks in schedules.items():
        print(f"P{agent+1}:")
        for task, start, end in tasks:
            print(f"  Task {task} from {start} to {end}")

Sorted tasks based on priority: ['1', '4', '3', '2', '5', '6', '9', '7', '8', '10']

Scheduling task: 1
Task 1 successors: []
Start time for task 1 on agent 0: 0
Finish time for task 1 on agent 0: 14
Best agent updated to 0 with earliest finish time 14
Start time for task 1 on agent 1: 0
Finish time for task 1 on agent 1: 16
Start time for task 1 on agent 2: 0
Finish time for task 1 on agent 2: 9
Best agent updated to 2 with earliest finish time 9
Task 1 scheduled on agent 2 from 0 to 9
Agent 2 available at 9

Scheduling task: 4
Task 4 successors: ['1']
Comm time between task 4 on agent 0 and task 1 on agent 2: 9
Start time for task 4 on agent 0: 18
Finish time for task 4 on agent 0: 31
Best agent updated to 0 with earliest finish time 31
Comm time between task 4 on agent 1 and task 1 on agent 2: 9
Start time for task 4 on agent 1: 18
Finish time for task 4 on agent 1: 26
Best agent updated to 1 with earliest finish time 26
Comm time between task 4 on agent 2 and task 1 on agent 2: 0
S

In [5]:
# # DAG 그래프 그리기 함수
# def draw_dag(dag):
#     G = nx.DiGraph(dag)
#     pos = nx.spring_layout(G)
#     plt.figure(figsize=(16, 8))
#     nx.draw(G, pos, with_labels=True, node_size=3000, node_color="lightblue", font_size=10, font_weight="bold", arrowsize=20)
#     plt.title("DAG Representation of the Model Architecture")
#     plt.show()

# # DAG 그래프 그리기
# draw_dag(dag)